In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook

In [2]:
import sys
sys.path.append("../../open-cd/")
sys.path.append("../../odeon/")

In [4]:
from mmengine import Registry, build_from_cfg

In [5]:
from mmengine import Registry
MODELS = Registry('opencd.models')

10/23 13:20:29 - mmengine - WARNING - set scope as "mmengine" when scope can not be inferred. You can silence this warning by passing a "scope" argument to Registry like `Registry(name, scope="toy")`


In [7]:
#from mmengine.registry import MODELS as MMENGINE_MODELS
#from mmengine.registry import Registry
import opencd.models

In [8]:
from opencd.registry import MODELS

In [9]:
from opencd.registry import RUNNERS

In [10]:
from mmengine.config import Config
cfg = Config.fromfile("../../open-cd/configs/changeformer/changeformer_mit-b1_256x256_40k_levircd.py")
cfg.work_dir = "./"

In [ ]:
build_from_cfg(cfg.model, MODELS)

In [8]:
data_preprocessor = MODELS.build(cfg.data_preprocessor)

In [11]:
runner = RUNNERS.build(cfg)

10/23 13:20:35 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
    CUDA available: False
    numpy_random_seed: 1507874261
    GCC: gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.0
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.1-Product Build 20220311 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CUDA_VERSION=11.7, CUDNN_VERSION=8.5.0, CXX_COMPILER=/opt/rh/devtoolset-9/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=0 -fabi-version=11 -Wno-depre

10/23 13:20:36 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
10/23 13:20:36 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/mmseg/models/backbones/mit.py:365: UserWarning: DeprecationWarning: pretrained is deprecated, please use "init_cfg" instead
  warnings.warn('DeprecationWarning: pretrained is deprecated, '
/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/mmseg/models/decode_heads/decode_head.py:120: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert `seg_logits` into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/mmseg/models/losses/cross_entropy_loss.py:249: Us

In [8]:
runner.model

SiamEncoderDecoder(
  (data_preprocessor): BaseDataPreprocessor()
  (backbone): MixVisionTransformer(
    (layers): ModuleList(
      (0): ModuleList(
        (0): PatchEmbed(
          (projection): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
          (norm): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
        )
        (1): ModuleList(
          (0-1): 2 x TransformerEncoderLayer(
            (norm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
            (attn): EfficientMultiheadAttention(
              (attn): MultiheadAttention(
                (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
              )
              (proj_drop): Dropout(p=0.0, inplace=False)
              (dropout_layer): DropPath()
              (sr): Conv2d(64, 64, kernel_size=(8, 8), stride=(8, 8))
              (norm): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
            )
            (norm2): LayerNorm((64,), ep

In [12]:
levir_root = "/mnt/store_dai/datasrc/dchan/levir-cd"
final_levir_root = "/mnt/stores/store-DAI/datasrc/dchan/levir-cd"

import glob
from pathlib import Path
import csv
from matplotlib import pyplot as plt
#plt.ioff()

import odeon
from odeon.data.data_module import Input

levir_path = Path(levir_root)
final_levir_path = Path(final_levir_root)

batch_size = 10
num_workers = 8

train_params = {
    'input_fields': {
        "T0": {"name": "A", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3]},
        "T1": {"name": "B", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3]},
        "mask": {"name": "label", "type": "mask", "encoding": "integer"}
    },
    'dataloader_options' : {"batch_size": batch_size, "num_workers": num_workers},
    'input_file': levir_path / "train.csv",
    'input_files_has_header': 'infer',
    'root_dir': levir_path,
    'nb_samples': 0,
    'sample_seed': 0
}

val_params = {
    'input_fields': {
        "T0": {"name": "A", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3]},
        "T1": {"name": "B", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3]},
        "mask": {"name": "label", "type": "mask", "encoding": "integer"}
    },
    'dataloader_options' : {"batch_size": batch_size, "num_workers": num_workers},
    'input_file': levir_path / "val.csv",
    'input_files_has_header': 'infer',
    'root_dir': levir_path,
    'nb_samples': 0,
    'sample_seed': 0
}

test_params = {
    'input_fields': {
        "T0": {"name": "A", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3]},
        "T1": {"name": "B", "type": "raster", "dtype": "uint8", "band_indices": [1, 2, 3]},
        "mask": {"name": "label", "type": "mask", "encoding": "integer"}
    },
    'dataloader_options' : {"batch_size": batch_size, "num_workers": num_workers},
    'input_file': levir_path / "test.csv",
    'root_dir': levir_path,
    'input_files_has_header': 'infer',
    'nb_samples': 0,
    'sample_seed': 0
}


input = Input(
    fit_params=train_params,
    validate_params=val_params,
    test_params=test_params
)


In [13]:
input.prepare_data()
input.setup(stage="fit")

In [14]:
v = input.fit.dataset[0]


/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [15]:
import torch
c = torch.cat((v['T0'], v['T1']), dim=0).unsqueeze(dim=0)
print(c.shape)
o = runner.model(c, v["mask"], mode="tensor")
o.shape

torch.Size([1, 6, 1024, 1024])


torch.Size([1, 2, 256, 256])

In [40]:
runner.build_optim_wrapper(cfg.optim_wrapper)

10/23 10:41:35 - mmengine - WARNING - Failed to search registry with scope "opencd" in the "optimizer wrapper constructor" registry tree. As a workaround, the current "optimizer wrapper constructor" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "opencd" is a correct scope, or whether the registry is initialized.
10/23 10:41:35 - mmengine - WARNING - Failed to search registry with scope "opencd" in the "optimizer" registry tree. As a workaround, the current "optimizer" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "opencd" is a correct scope, or whether the registry is initialized.
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.0.0.norm.weight:lr=6e-05
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.0.0.norm.weight:weight_decay=0.0
10/23 10:41:35 - mmengine - 

10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.1.1.0.norm2.bias:lr=6e-05
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.1.1.0.norm2.bias:weight_decay=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.1.1.0.norm2.bias:decay_mult=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.1.1.1.norm1.weight:lr=6e-05
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.1.1.1.norm1.weight:weight_decay=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.1.1.1.norm1.weight:decay_mult=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.1.1.1.norm1.bias:lr=6e-05
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.1.1.1.norm1.bias:weight_decay=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.1.1.1.norm1.bias:decay_mult=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layer

10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.3.1.0.norm1.weight:weight_decay=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.3.1.0.norm1.weight:decay_mult=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.3.1.0.norm1.bias:lr=6e-05
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.3.1.0.norm1.bias:weight_decay=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.3.1.0.norm1.bias:decay_mult=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.3.1.0.norm2.weight:lr=6e-05
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.3.1.0.norm2.weight:weight_decay=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.3.1.0.norm2.weight:decay_mult=0.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.layers.3.1.0.norm2.bias:lr=6e-05
10/23 10:41:35 - mmengine - INFO - paramwise_options -- backbone.l

10/23 10:41:35 - mmengine - INFO - paramwise_options -- decode_head.fusion_conv.bn.weight:lr_mult=10.0
10/23 10:41:35 - mmengine - INFO - paramwise_options -- decode_head.fusion_conv.bn.bias:lr=0.0006000000000000001
10/23 10:41:35 - mmengine - INFO - paramwise_options -- decode_head.fusion_conv.bn.bias:weight_decay=0.01
10/23 10:41:35 - mmengine - INFO - paramwise_options -- decode_head.fusion_conv.bn.bias:lr_mult=10.0
10/23 10:41:35 - mmengine - WARNING - Failed to search registry with scope "opencd" in the "optim_wrapper" registry tree. As a workaround, the current "optim_wrapper" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "opencd" is a correct scope, or whether the registry is initialized.


Type: OptimWrapper
_accumulative_counts: 1
optimizer: 
AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 6e-05
    maximize: False
    weight_decay: 0.01

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 6e-05
    maximize: False
    weight_decay: 0.01

Parameter Group 2
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decay_mult: 0.0
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 6e-05
    maximize: False
    weight_decay: 0.0

Parameter Group 3
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    decay_mult: 0.0
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 6e-05
    maximize: False
    weight_decay: 0.0

Parameter Group 4
    amsgrad: Fals

In [17]:
import odeon
from odeon.models.opencd import OpenCDPlug

In [19]:
ocd = OpenCDPlug(
    config_file="../../open-cd/configs/changeformer/changeformer_mit-b1_256x256_40k_levircd.py",
    loss="ce"
)

Executor open_cd_plug already exists. Will replace it
[autoreload of odeon.models.opencd failed: Traceback (most recent call last):
  File "/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/PVoitot/mambaforge/envs/odeon/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/mnt/store_dai/equipiers/pvoitot/dchan/odeon/notebooks/../../odeon/odeon/models/opencd.py", line 47, in <module>
    class OpenCDPlug(pl.LightningModule):
  File "/mnt/store_dai

KeyError: 'work_dir'